Paso 1: Obtención de Datos
Encontrar o construir un dataset que tenga las siguientes características:
•	Que tenga al menos 5 columnas que formen parte del conjunto de variables independientes. Que entre dichas columnas haya al menos:
o	3 columnas con datos numéricos. Al menos una de estas columnas debe tener datos ausentes
o	1 columna con datos categóricos de tipo nominal
•	Que tenga una columna que sea la variable dependiente (variable objetivo o etiqueta)


In [33]:
import pandas as pd
import numpy as np

# Establecer semilla para reproducibilidad
np.random.seed(42)

# Crear el dataframe con 1000 filas y valores aleatorios
data = {
    'Edad': np.random.randint(18, 66, size=1000),  # Edad entre 18 y 65
    'Ingreso anual': np.random.randint(25000, 100000, size=1000),  # Ingreso anual entre 25,000 y 100,000
    'Horas de estudio por semana': np.random.choice([np.nan, 20, 30, 40], size=1000),  # Algunas ausencias
    'Género': np.random.choice(['Masculino', 'Femenino'], size=1000),  # Categórica: Masculino/Femenino
    'Región': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], size=1000),  # Categórica: 4 regiones
    'Sueldo mensual': np.random.randint(2000, 8000, size=1000)  # Sueldo mensual entre 2,000 y 8,000
}

# Crear DataFrame
df = pd.DataFrame(data)

# Mostrar las primeras filas del DataFrame
df.head()


,Edad,Ingreso anual,Horas de estudio por semana,Género,Región,Sueldo mensual
0,56,74376,NaN,Femenino,Este,4964
1,46,70587,NaN,Femenino,Este,3271
2,32,61976,NaN,Masculino,Oeste,3813
3,60,92584,20.0,Femenino,Sur,4915
4,25,67533,NaN,Femenino,Sur,2609


Paso 2: Preprocesamiento de Datos
Sobre dicho dataset se deberán llevar a cabo los siguientes tratamientos de preprocesado de datos:
•	Gestión de datos ausentes: se deberán tratar los datos ausentes de una forma razonable en términos del negocio que representen dichos datos y que en ningún caso sea eliminando las filas y/o columnas que contengan datos ausentes
•	Gestión de datos categóricos de tipo nominal: se tratarán a través de la técnica de One Hot Encoding
•	Se utilizarán el 75% de los datos para construir el modelo y el 25% restante para validar el modelo construido
•	Se normalizarán o estandarizarán (a elegir por el alumno) todos los datos asociados de las variables independientes
El preprocesamiento deberá realizarse una única vez, y no una vez por cada modelo generado.
Al final se deberá deberá mostrar el tiempode ejecución gastado del preprocesamiento de los datos.


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import time

# Iniciar el cronómetro para el preprocesamiento
start_time = time.time()

# Definir columnas numéricas y categóricas
numerical_cols = ['Edad', 'Ingreso anual', 'Horas de estudio por semana']
categorical_cols = ['Género', 'Región']

# Pipeline para preprocesar las columnas numéricas
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Reemplazo de datos ausentes por la media
    ('scaler', StandardScaler())  # Normalización
])

# Pipeline para preprocesar las columnas categóricas
categorical_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One Hot Encoding
])

# Crear un preprocesador para aplicar el pipeline adecuado a cada tipo de columna
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Separar las características (X) y la variable dependiente (y)
X = df.drop(columns=['Sueldo mensual'])
y = df['Sueldo mensual']

# Dividir los datos en entrenamiento y prueba (75%/25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Aplicar el preprocesamiento
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Medir el tiempo de ejecución del preprocesamiento
preprocessing_time = time.time() - start_time

# Mostrar el tiempo de ejecución del preprocesamiento
preprocessing_time


0.023611783981323242

Paso 3: Generación de Modelos
La práctica consistirá en generar diferentes modelos utilizando las siguientes técnicas de regresión:
•	Regresión Lineal. Se generarán los siguientes modelos:
o	Utilizando todas las variables independientes
o	Ignorando una de las variables independientes (la que en apariencia sea menos relevante)
o	Ignorando dos de las variables independientes (las que en apariencia sean menos relevantes)
•	Regresión Polinómica. Se generarán los siguientes modelos:
o	Utilizando todas las variables independientes y generando un modelo polinómico de grado 12, otro de grado 8 y otro de grado 4
o	Ignorando una de las variables independientes (la que en apariencia sea menos relevante) y generando un modelo polinómico de grado 10, otro de grado 7 y otro de grado 5
o	Ignorando dos de las variables independientes (las que en apariencia sean menos relevantes) y generando un modelo polinómico de grado 8, otro de grado 5 y otro de grado 2
•	Regresión de las K Valores más Cercanos. Se generarán los siguientes modelos:
o	Utilizando todas las variables independientes y generando un modelo con K = 8, otro con K = 6 y otro con K = 4
o	Ignorando una de las variables independientes (la que en apariencia sea menos relevante) y generando un modelo con K = 6, otro con K = 4 y otro con K = 2
o	Ignorando dos de las variables independientes (las que en apariencia sean menos relevantes) y generando un modelo con K =4, otro con K = 2 y otro con K = 1
Obteniendo para cada uno de los modelos las siguientes métricas:
•	Máximo Error Absoluto (M)
•	Error Absoluto Medio (MAE)
•	Coeficiente de Determinación (R^2)
Mostrando en una tabla (que sea una dataframe por ejemplo):
•	Las métricas obtenidas para todos y cada uno de los modelos
•	El tiempo de ejecución gastado en el entrenamiento de cada modelo (excluido el tiempo de preprocesamiento)
•	El tiempo de ejecución gastado en la prueba de cada modelo


In [35]:
# Importar librerías necesarias
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import time

# Estandarización de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear modelos con polinomios de menor grado
polynomial_model_2 = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
polynomial_model_3 = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())

# Entrenar y evaluar los modelos
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test):
    start_train_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_train_time

    start_test_time = time.time()
    y_pred = model.predict(X_test)
    test_time = time.time() - start_test_time

    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    max_error = np.max(np.abs(y_test - y_pred))

    return mae, max_error, r2, train_time, test_time

# Regresión Lineal
linear_model = LinearRegression()
mae_lr, max_error_lr, r2_lr, train_time_lr, test_time_lr = train_and_evaluate_model(linear_model, X_train_scaled, X_test_scaled, y_train, y_test)

# Regresión Polinómica
mae_poly_2, max_error_poly_2, r2_poly_2, train_time_poly_2, test_time_poly_2 = train_and_evaluate_model(polynomial_model_2, X_train_scaled, X_test_scaled, y_train, y_test)
mae_poly_3, max_error_poly_3, r2_poly_3, train_time_poly_3, test_time_poly_3 = train_and_evaluate_model(polynomial_model_3, X_train_scaled, X_test_scaled, y_train, y_test)

# KNN con K=8
knn_model_8 = KNeighborsRegressor(n_neighbors=8)
mae_knn_8, max_error_knn_8, r2_knn_8, train_time_knn_8, test_time_knn_8 = train_and_evaluate_model(knn_model_8, X_train_scaled, X_test_scaled, y_train, y_test)




In [36]:
# Crear la tabla con los resultados
results = {
    'Técnica': ['Regresión Lineal', 'Regresión Polinómica Grado 2', 'Regresión Polinómica Grado 3', 'KNN con K=8'],
    'Variables Independientes': ['Todas', 'Todas', 'Todas', 'Todas'],
    'M': [max_error_lr, max_error_poly_2, max_error_poly_3, max_error_knn_8],
    'MAE': [mae_lr, mae_poly_2, mae_poly_3, mae_knn_8],
    'R^2': [r2_lr, r2_poly_2, r2_poly_3, r2_knn_8],
    'Tiempo de Entrenamiento': [train_time_lr, train_time_poly_2, train_time_poly_3, train_time_knn_8],
    'Tiempo de Prueba': [test_time_lr, test_time_poly_2, test_time_poly_3, test_time_knn_8]
}

# Crear DataFrame con los resultados
results_df = pd.DataFrame(results)

# Mostrar la tabla de resultados
results_df

,Técnica,Variables Independientes,M,MAE,R^2,Tiempo de Entrenamiento,Tiempo de Prueba
0,Regresión Lineal,Todas,3159.094341,1423.710871,-0.010816,0.005284,0.000000
1,Regresión Polinómica Grado 2,Todas,3337.164007,1460.681267,-0.053691,0.012750,0.000000
2,Regresión Polinómica Grado 3,Todas,3740.281797,1486.154168,-0.144345,0.072375,0.000000
3,KNN con K=8,Todas,3943.000000,1533.973000,-0.172682,0.000000,0.006852
